In [21]:
!pip install accelerate

In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig,AutoModelForCausalLM
import pandas as pd
import random
from tqdm import tqdm

In [2]:
import torch

In [3]:
config = AutoConfig.from_pretrained("neulab/omnitab-large-finetuned-wtq")
# IMPORTANT: the initial BART model's decoding is penalized by no_repeat_ngram_size, and thus
# we should disable it here to avoid problematic generation
config.no_repeat_ngram_size = 3 # В omnitab = 3, part exec = 0
config.max_length = 1024
#config.early_stopping = False #  В омнитаб нет

#model = AutoModel.from_config(config) # Для создания экземпляра модели без предобученных весов
tokenizer = AutoTokenizer.from_pretrained('neulab/omnitab-large-finetuned-wtq')
model = AutoModelForSeq2SeqLM.from_pretrained('neulab/omnitab-large-finetuned-wtq')
#model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", device_map = 'auto')
device = "cpu"#"cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [4]:
data = {
    'year': [1896, 1900, 1904, 2004, 2008, 2012],
    'city': ['athens', 'paris', 'st. louis', 'athens', 'beijing', 'london']
}
table = pd.DataFrame.from_dict(data)

query = 'In which year did beijing host the Olympic Games?'
encoding = tokenizer(table=table, query=query, return_tensors='pt')

outputs = model.generate(**encoding)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

[' 2008']


In [5]:
next(model.parameters()).device

device(type='cpu')

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [8]:
from datasets import load_dataset


In [44]:
data_files = {"train":"data/wtq_lf_preorder/train/data-00000-of-00001.arrow",
              "test":"data/wtq_lf_preorder/test/data-00000-of-00001.arrow",
              "validation":"data/wtq_lf_preorder/validation/data-00000-of-00001.arrow"}
datasets = load_dataset("arrow", data_files=data_files)
#datasets = load_dataset("Stanford/wikitablequestions")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 8610
    })
    test: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 4344
    })
    validation: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 2092
    })
})

In [45]:
datasets = datasets["test"]
datasets

Dataset({
    features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
    num_rows: 4344
})

In [11]:
target_col_name = f"lf_pcs"


In [ ]:
#features = ["question","table"]
#def length_filter(example):
#    return {f"length_{target_col_name}": len(tokenizer.tokenize(example[target_col_name]))<=max_target_length}
#
#for mode in ['train',"validation"]:#
#
#   if mode =="train":
#        datasets[mode] = datasets[mode].map(length_filter)
#        filter_col = [idx for idx, i in enumerate(datasets[mode][filter_col_name]) if i==1]
#        logger.info(f"remove {len(datasets[mode])-len(filter_col)} observations with length > {max_target_length}.")
#        logger.info(f"Previous {mode} size {len(datasets[mode])} actual {mode} size {len(filter_col)}")
#        datasets[mode] = datasets[mode].select(filter_col)
#
#    removed_cols = set(datasets[mode].features.keys())-set([target_col_name]+features)
#    datasets[mode] = datasets[mode].remove_columns(removed_cols)
#    datasets[mode] = datasets[mode].rename_column(target_col_name, 'answers')

In [12]:
from data_processing.metrics import (to_value_list,
                                     to_value,
                                     plot_and_save_model_performance,
                                     fuzzy_matching,
                                     strict_denotation_accuracy,
                                     flexible_denotation_accuracy)

/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<ipykernel.iostream.OutStream object at 0x7f3f1d6b0460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-seen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7f3f1d6b0460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-unseen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7f3f1d6b0460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/training.tagged


In [14]:
path_model_folder = "./models/OmniTab"
output_file = f'{path_model_folder}/model_evaluation_results.json'
#target_col_name = f"lf_{Omega_include.lower()}"
#Omega_include = path_model_folder.split('/')[-1].split('_')[1]

In [ ]:
#removed_cols = set(datasets["validation"].features.keys())-set([target_col_name]+features+["answers"])
#datasets["validation"] = datasets["validation"].remove_columns(removed_cols)
#datasets["validation"] = datasets["validation"].rename_column(target_col_name, 'answers_lf')
#datasets["validation"] = datasets["validation"].rename_column("answers", 'answers_exec')
#datasets["test"] = datasets["test"].rename_column("answers", 'answers_exec')

In [21]:
rnd = random.randint(0, len(datasets))

In [26]:
rnd

1758

In [66]:
def preprocess_tableqa_function(example, is_test=False):

        question = example["question"].lower()
        example_table = example["table"]
        table = pd.DataFrame.from_records(example_table["rows"], columns=example_table["header"])
        
        model_input = tokenizer(
            table=table, query=question, max_length=1024, padding=False, truncation=True
        )
            
        return model_input


In [75]:
def evaluate_model(model, dataset, comp_graph = False, is_test=False,flatten_mode="preorder"):
        model.eval()
        perfs={ "Strict_Denotation_Accuracy_Execs":[]}
        for idx, example in tqdm(enumerate(dataset), total=len(dataset)):
            proces_example = preprocess_tableqa_function(example)
            input_ids = proces_example["input_ids"]
            attention_mask = proces_example["attention_mask"]

            input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
            attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            prediction_lf = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True).strip()
            if comp_graph:
                header=tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)
                header=header.split(' col : ')[1].split(' row 1 : ')[0].split(' |')
                header=[h.strip() for h in header]
                #G=parse(prediction_lf, header, flatten_mode=args.flatten_mode)
                try:
                    print('1')
                    G=parse(prediction_lf, header, flatten_mode=flatten_mode)
                    print('2')
                    prediction_exec = G.executed_last_node()
                    print(f'answers_exec {answers_exec} prediction_exec {prediction_exec}')
                    acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list(prediction_exec))

                except Exception as e:
                    print(e)
                    acc_ex_stric = False

            
            else:
                prediction_exec = prediction_lf
                answers_exec = example['answers']
                print(prediction_exec,answers_exec)
                acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list([prediction_exec]))
            print(f'Strict_Denotation_Accuracy_Execs {acc_ex_stric}')
            perfs["Strict_Denotation_Accuracy_Execs"].append(acc_ex_stric)

        Strict_Denotation_Accuracy_Exec = sum(perfs["Strict_Denotation_Accuracy_Execs"])/len(perfs["Strict_Denotation_Accuracy_Execs"])

        return {'Strict_Denotation_Accuracy_Exec': Strict_Denotation_Accuracy_Exec}  # example


In [ ]:
evaluate_model(model,datasets)

  0%|          | 1/4344 [00:00<22:22,  3.24it/s]

Spain ['Italy']
Strict_Denotation_Accuracy_Execs False


  0%|          | 2/4344 [00:00<23:47,  3.04it/s]

100,000 ['100,000']
Strict_Denotation_Accuracy_Execs True


  0%|          | 3/4344 [00:01<28:40,  2.52it/s]

...

  4%|▍         | 185/4344 [01:13<31:43,  2.18it/s]

Eddie Johnson ['Eddie Johnson']
Strict_Denotation_Accuracy_Execs True


  4%|▍         | 186/4344 [01:13<27:02,  2.56it/s]

United States ['United States']
Strict_Denotation_Accuracy_Execs True


  4%|▍         | 187/4344 [01:13<25:27,  2.72it/s]

In [67]:
preprocess_tableqa_function(datasets[0])


{'input_ids': [0, 5488, 247, 56, 5, 144, 19294, 2073, 624, 5, 299, 158, 116, 11311, 4832, 17816, 1721, 25544, 661, 1721, 2711, 1721, 3421, 1721, 7437, 100, 1698, 31814, 37457, 282, 46340, 3236, 112, 4832, 112, 1721, 8445, 25787, 3767, 2802, 2794, 50141, 1640, 1723, 510, 43, 1721, 8316, 8035, 385, 108, 39318, 5384, 858, 1721, 195, 298, 1132, 108, 158, 113, 1721, 843, 3236, 132, 4832, 132, 1721, 21441, 338, 16052, 2413, 858, 705, 50141, 1640, 500, 3048, 43, 1721, 2711, 230, 3632, 22506, 139, 788, 1721, 579, 4, 90, 4, 1721, 389, 3236, 155, 4832, 155, 1721, 7234, 1949, 47045, 179, 50141, 1640, 2068, 250, 43, 1721, 7965, 1168, 7864, 254, 1721, 579, 4, 90, 4, 1721, 564, 3236, 204, 4832, 204, 1721, 5476, 5675, 23479, 2531, 50141, 1640, 2068, 250, 43, 1721, 13287, 10577, 1721, 579, 4, 90, 4, 1721, 291, 3236, 195, 4832, 195, 1721, 14185, 29755, 1210, 14683, 50141, 1640, 2068, 250, 43, 1721, 26185, 1023, 281, 1721, 579, 4, 90, 4, 1721, 379, 3236, 231, 4832, 231, 1721, 16439, 4011, 611, 1417, 501

In [72]:
datasets[0]["answers"]

['Italy']